In [1]:
""" This script tries only using the TuLiP toolbox to solve a multirobot planning execution (MrPE) problem.
A predefined paths are given and a control policy is generated by this script
"""

# Import the packages that we need
from __future__ import print_function
import numpy as np
from tulip import spec
from tulip.transys import machines
from tulip import synth
import gr1_fragment

`omega.symbolic.symbolic` failed to import `dd.cudd`.
Will use `dd.autoref`.


In [2]:
# Define the parameters & desired trajectories
num_robots = 2  # must be > 1
num_cells = 5
trajectories = [[1, 2, 3], [4, 2, 5]]

In [3]:
# Define the env_vars based on the params % trajectories
env_vars = set()
env_vars_list = []
visit_times_all = np.zeros(num_cells, dtype=int)
for r in range(1, num_robots + 1):
    env_vars |= {'init' + str(r)}
    env_vars_list.append([])
    visit_times = np.zeros(num_cells, dtype=int)
    for p in trajectories[r - 1]:
        visit_times[p - 1] += 1
        new_env_var = 's' + str(r) + 'a' + str(p) + 'a' + str(visit_times[p - 1])
        env_vars |= {new_env_var}
        env_vars_list[r - 1].append(new_env_var)
    visit_times_all = np.row_stack((visit_times_all, visit_times))
visit_times_all = np.delete(visit_times_all, 0, 0)

In [4]:
print(env_vars)

{'s2a2a1', 'init1', 's1a1a1', 's1a3a1', 's1a2a1', 'init2', 's2a4a1', 's2a5a1'}


In [5]:
# Define the initial conditions
env_init = set()
for r in range(1, num_robots + 1):
    env_init |= {'init' + str(r)}

In [6]:
print(env_init)

{'init2', 'init1'}


In [7]:
env_safe = set()
env_prog = set()
# Encode the constraint that each robot can only appear in one position
env_unique = set()
for r in range(1, num_robots + 1):
    new_env_unique = '( ' + 'init' + str(r) + ' ) <-> ( '
    for p in env_vars_list[r - 1]:
        new_env_unique += '!' + p + ' && '
    new_env_unique = new_env_unique[:-3] + ')'
    env_unique |= {new_env_unique}
for r in range(1, num_robots + 1):
    for i in range(len(env_vars_list[r - 1])):
        new_env_unique = '( ' + env_vars_list[r - 1][i] + ' ) <-> ( ' + '!' + 'init' + str(r)
        for j in range(len(env_vars_list[r - 1])):
            if j != i:
                new_env_unique += ' && !' + env_vars_list[r - 1][j]
        new_env_unique += ' )'
        env_unique |= {new_env_unique}
env_safe |= env_unique

In [8]:
print(env_unique)

{'( s2a2a1 ) <-> ( !init2 && !s2a4a1 && !s2a5a1 )', '( init1 ) <-> ( !s1a1a1 && !s1a2a1 && !s1a3a1 )', '( s2a5a1 ) <-> ( !init2 && !s2a4a1 && !s2a2a1 )', '( s2a4a1 ) <-> ( !init2 && !s2a2a1 && !s2a5a1 )', '( s1a3a1 ) <-> ( !init1 && !s1a1a1 && !s1a2a1 )', '( init2 ) <-> ( !s2a4a1 && !s2a2a1 && !s2a5a1 )', '( s1a1a1 ) <-> ( !init1 && !s1a2a1 && !s1a3a1 )', '( s1a2a1 ) <-> ( !init1 && !s1a1a1 && !s1a3a1 )'}


In [9]:
# Encode the transition system relations
env_trans = set()
# First, from initial positions to the first cells
for r in range(1, num_robots + 1):
    env_trans |= {'( ' + 'init' + str(r) + ' && ' + 'go' + str(r) + ' ) -> X( ' + 'init' + str(r) + ' || ' +
                  env_vars_list[r - 1][0] + ' )'}
    env_trans |= {'( ' + 'init' + str(r) + ' && ' + '!go' + str(r) + ' ) -> X( ' + 'init' + str(r) + ' )'}
# Then, from cells to cells
for r in range(1, num_robots + 1):
    for i in range(len(env_vars_list[r - 1]) - 1):
        env_trans |= {'( ' + env_vars_list[r - 1][i] + ' && ' + 'go' + str(r) + ' ) -> X( ' + env_vars_list[r - 1][i]
                      + ' || ' + env_vars_list[r - 1][i + 1] + ' )'}
        env_trans |= {'( ' + env_vars_list[r - 1][i] + ' && ' + '!go' + str(r) + ' ) -> X( ' + env_vars_list[r - 1][i]
                      + ' )'}
    env_trans |= {'( ' + env_vars_list[r - 1][i+1] + ' ) -> X( ' + env_vars_list[r - 1][i+1]
                  + ' )'}
env_safe |= env_trans

In [10]:
print(env_trans)

{'( s2a2a1 && !go2 ) -> X( s2a2a1 )', '( s2a4a1 && go2 ) -> X( s2a4a1 || s2a2a1 )', '( s1a2a1 && go1 ) -> X( s1a2a1 || s1a3a1 )', '( s2a2a1 && go2 ) -> X( s2a2a1 || s2a5a1 )', '( s1a1a1 && go1 ) -> X( s1a1a1 || s1a2a1 )', '( s1a2a1 && !go1 ) -> X( s1a2a1 )', '( init2 && go2 ) -> X( init2 || s2a4a1 )', '( init1 && go1 ) -> X( init1 || s1a1a1 )', '( init2 && !go2 ) -> X( init2 )', '( s2a5a1 ) -> X( s2a5a1 )', '( init1 && !go1 ) -> X( init1 )', '( s1a1a1 && !go1 ) -> X( s1a1a1 )', '( s2a4a1 && !go2 ) -> X( s2a4a1 )', '( s1a3a1 ) -> X( s1a3a1 )'}


In [11]:
# There is no such assumption on the environment.
# Encode the constraint that every robot should reach its destination and stay
env_dest = set()
sys_dest_stop = set()
for r in range(1, num_robots + 1):
    env_dest |= {'( ' + env_vars_list[r - 1][-1] + ' ) -> X( ' + env_vars_list[r - 1][-1] + ' )'}
    sys_dest_stop |= {env_vars_list[r - 1][-1]}
# env_safe |= env_dest

In [12]:
print(env_dest)

{'( s2a5a1 ) -> X( s2a5a1 )', '( s1a3a1 ) -> X( s1a3a1 )'}


In [13]:
print(sys_dest_stop)

{'s2a5a1', 's1a3a1'}


In [14]:
# Encode the constraint that each robot should either not go or eventually not in the current cell
#env_go = set()
#for r in range(1, num_robots + 1):
#    env_go |= {'( init' + str(r) + ' && go' + str(r) + ' && X(!init' + str(r) +') )'}
#    for i in range(len(env_vars_list[r - 1]) - 1):
#        env_go |= {'(' + env_vars_list[r - 1][i] + ' && ' + 'go' + str(r) + ' && X(!'+ env_vars_list[r - 1][i]+ ') )'}
#env_prog |= env_go

# Encode the constraint that each robot should either not go or eventually not in the current cell
env_go = set()
for r in range(1, num_robots + 1):
    env_go |= {'( !' + 'init' + str(r) + ' || ' + '!go' + str(r) + ' )'}
    for i in range(len(env_vars_list[r - 1]) - 1):
        env_go |= {'( !' + env_vars_list[r - 1][i] + ' || ' + '!go' + str(r) + ' )'}
env_prog |= env_go

In [15]:
print(env_go)

{'( !init1 || !go1 )', '( !init2 || !go2 )', '( !s2a4a1 || !go2 )', '( !s1a1a1 || !go1 )', '( !s2a2a1 || !go2 )', '( !s1a2a1 || !go1 )'}


In [16]:
# Define sys_vars
sys_vars = set()
for r in range(1, num_robots + 1):
    sys_vars |= {'go' + str(r)}

In [17]:
print(sys_vars)

{'go1', 'go2'}


In [18]:
# Define initial command. Now assume each robot is stopped at the beginning
sys_init = set()
# for r in range(1, num_robots + 1):
#     sys_init |= {'!go' + str(r)}

sys_safe = set()
sys_prog = set()

In [19]:
'''
dest_stop_str = ''
for i in sys_dest_stop:
    dest_stop_str += i + ' && '
dest_stop_str = dest_stop_str[:-4]
spec_dest = gr1_fragment.stability_to_gr1(dest_stop_str, 'dest_aux')
sys_vars |= {'dest_aux'}
sys_init |= set(spec_dest.sys_init)
sys_safe |= set(spec_dest.sys_safety)
sys_prog |= set(spec_dest.sys_prog)
'''
sys_prog |= sys_dest_stop  # not working

In [20]:
# Encode the constraint that if ending point is reached, not go. And if ending point is not reached, always exist some
# robot that goes
sys_stop = set()
new_sys_nonstop = '( '
for r in range(1, num_robots + 1):
    sys_stop |= {'( ' + env_vars_list[r - 1][-1] + ' ) -> ( ' + '!go' + str(r) + ' )'}
    new_sys_nonstop += '( !' + env_vars_list[r - 1][-1] + ' ) || '
new_sys_nonstop = new_sys_nonstop[:-3] + ') -> ( '
for r in range(1, num_robots + 1):
    new_sys_nonstop += 'go' + str(r) + ' || '
new_sys_nonstop = new_sys_nonstop[:-3] + ')'
sys_stop |= {new_sys_nonstop}
# sys_safe |= sys_stop  # not working

In [21]:
# Encode the collision avoidance constraint
sys_collision = set()  # A robot cannot go if the next position is currently occupied
for r in range(1, num_robots + 1):
    # From initial
    new_sys_collision = '( ' + 'init' + str(r) + ' && ( '
    next_pos_from_init = trajectories[r - 1][0]
    collision_possible = False
    for other_r_init in range(1, num_robots + 1):
        if other_r_init != r:
            for j in range(visit_times_all[other_r_init - 1][next_pos_from_init - 1]):
                new_sys_collision += 's' + str(other_r_init) + 'a' + str(next_pos_from_init) + 'a' + str(j + 1) + ' || '
                collision_possible = True
    if collision_possible:
        new_sys_collision = new_sys_collision[:-3] + ') ) -> !go' + str(r)
        sys_collision |= {new_sys_collision}
    # From cells to cells
    for pos_index in range(len(env_vars_list[r - 1]) - 1):
        new_sys_collision = '( ' + env_vars_list[r - 1][pos_index] + ' && ( '
        next_pos = trajectories[r - 1][pos_index + 1]
        collision_possible = False
        for other_r in range(1, num_robots + 1):
            if other_r != r:
                for j in range(visit_times_all[other_r - 1][next_pos - 1]):
                    new_sys_collision += 's' + str(other_r) + 'a' + str(next_pos) + 'a' + str(j + 1) + ' || '
                    collision_possible = True
        if collision_possible:
            new_sys_collision = new_sys_collision[:-3] + ') ) -> !go' + str(r)
            sys_collision |= {new_sys_collision}
#sys_safe |= sys_collision

In [22]:
print(sys_collision)

{'( s1a1a1 && ( s2a2a1 ) ) -> !go1', '( s2a4a1 && ( s1a2a1 ) ) -> !go2'}


In [23]:
sys_collision_2 = set()  # If 2 robots are going to the same cell theoretically at the next timestamp,
# at least one should not go
for cell in range(1, num_cells + 1):
    for r in range(1, num_robots + 1):
        for other_r in range(r + 1, num_robots + 1):
            index_1 = -1
            for i in range(visit_times_all[r - 1][cell - 1]):
                index_1_temp = trajectories[r - 1][(index_1 + 1):].index(cell)
                index_1 += index_1_temp + 1
                index_2 = -1
                for j in range(visit_times_all[other_r - 1][cell - 1]):
                    index_2_temp = trajectories[other_r - 1][(index_2 + 1):].index(cell)
                    index_2 += index_2_temp + 1
                    if index_1 == 0:
                        new_sys_collision_2 = '( init' + str(r) + ' && '
                    else:
                        new_sys_collision_2 = '( ' + env_vars_list[r - 1][index_1 - 1] + ' && '
                    if index_2 == 0:
                        new_sys_collision_2 += 'init' + str(other_r) + ' )'
                    else:
                        new_sys_collision_2 += env_vars_list[other_r - 1][index_2 - 1] + ' )'
                    new_sys_collision_2 += ' -> (!go' + str(r) + ' || ' + '!go' + str(other_r) + ' )'
                    sys_collision_2 |= {new_sys_collision_2}

#sys_safe |= sys_collision_2
sys_col_simple = {'!(s1a2a1 && s2a2a1)'}
sys_safe |= sys_col_simple

In [24]:
print(sys_collision_2)

{'( s1a1a1 && s2a4a1 ) -> (!go1 || !go2 )'}


In [28]:
# Create a GR(1) specification
specs = spec.GRSpec(env_vars, sys_vars, env_init, sys_init,
                    env_safe, sys_safe, env_prog, sys_prog)
specs.qinit = '\A \E'  
specs.moore = False
specs.plus_one = False

#print(specs.env_vars)
#print(specs.env_init)
print(specs.env_safety)
#print(specs.env_prog)
print(specs.sys_vars)
print(specs.sys_init)
print(specs.sys_safety)
print(specs.sys_prog)

['( s2a2a1 && !go2 ) -> X( s2a2a1 )', '( s1a3a1 ) <-> ( !init1 && !s1a1a1 && !s1a2a1 )', '( s1a1a1 && go1 ) -> X( s1a1a1 || s1a2a1 )', '( s1a2a1 ) <-> ( !init1 && !s1a1a1 && !s1a3a1 )', '( s1a2a1 && !go1 ) -> X( s1a2a1 )', '( s2a2a1 ) <-> ( !init2 && !s2a4a1 && !s2a5a1 )', '( init2 && !go2 ) -> X( init2 )', '( s2a5a1 ) -> X( s2a5a1 )', '( s1a1a1 && !go1 ) -> X( s1a1a1 )', '( s2a4a1 && go2 ) -> X( s2a4a1 || s2a2a1 )', '( s2a4a1 ) <-> ( !init2 && !s2a2a1 && !s2a5a1 )', '( s1a2a1 && go1 ) -> X( s1a2a1 || s1a3a1 )', '( s2a2a1 && go2 ) -> X( s2a2a1 || s2a5a1 )', '( init2 && go2 ) -> X( init2 || s2a4a1 )', '( init1 && go1 ) -> X( init1 || s1a1a1 )', '( init1 ) <-> ( !s1a1a1 && !s1a2a1 && !s1a3a1 )', '( init1 && !go1 ) -> X( init1 )', '( s2a5a1 ) <-> ( !init2 && !s2a4a1 && !s2a2a1 )', '( s1a3a1 ) -> X( s1a3a1 )', '( s2a4a1 && !go2 ) -> X( s2a4a1 )', '( init2 ) <-> ( !s2a4a1 && !s2a2a1 && !s2a5a1 )', '( s1a1a1 ) <-> ( !init1 && !s1a2a1 && !s1a3a1 )']
{'go1': 'boolean', 'go2': 'boolean'}
[]
['!

In [29]:
print('Start synthesis')
#ctrl = synth.synthesize(specs, solver='gr1c')
ctrl = synth.synthesize(specs)
print('End synthesis')
assert ctrl is not None, 'unrealizable'

Start synthesis
The initial condition requirement:
\A x:  \E y:  SysInit /\ (EnvInit => Win)
does not hold, so for some `EnvInit` states, the component cannot pick an initial `SysInit` state 
that is also winning.
End synthesis


AssertionError: unrealizable

In [27]:
machines.random_run(ctrl, N=30)

move from
	 state: Sinit
	 with input:{'s2a2a1': False, 'init1': True, 's1a1a1': False, 's1a3a1': False, 's1a2a1': False, 'init2': True, 's2a4a1': False, 's2a5a1': False}
	 to state: 0
	 reacting by producing output: {'go1': False, 'go2': False}
move from
	 state: 0
	 with input:{'s2a2a1': False, 'init1': True, 's1a1a1': False, 's1a3a1': False, 's1a2a1': False, 'init2': True, 's2a4a1': False, 's2a5a1': False}
	 to state: 64
	 reacting by producing output: {'go1': False, 'go2': True}
move from
	 state: 64
	 with input:{'s2a2a1': False, 'init1': True, 's1a1a1': False, 's1a3a1': False, 's1a2a1': False, 'init2': True, 's2a4a1': False, 's2a5a1': False}
	 to state: 64
	 reacting by producing output: {'go1': False, 'go2': True}
move from
	 state: 64
	 with input:{'s2a2a1': False, 'init1': True, 's1a1a1': False, 's1a3a1': False, 's1a2a1': False, 'init2': True, 's2a4a1': False, 's2a5a1': False}
	 to state: 64
	 reacting by producing output: {'go1': False, 'go2': True}
move from
	 state: 64
	 wi

([0,
  64,
  64,
  64,
  64,
  64,
  65,
  97,
  98,
  114,
  114,
  114,
  114,
  115,
  123,
  187,
  188,
  220,
  221,
  237,
  238,
  246,
  238,
  246,
  238,
  246,
  238,
  246,
  238,
  246],
 {'go1': [False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   True,
   False,
   True,
   False,
   True,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False],
  'go2': [False,
   True,
   True,
   True,
   True,
   True,
   False,
   True,
   False,
   True,
   True,
   True,
   True,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False]})